In [1]:
import os
import sys
import fnmatch
import pandas as pd
import operator
import shutil
import re
import javalang
def create_dev_test_train_split_and_vocabulary(root_path, 
                                               train_output, 
                                               vocabFile,
                                               combine
                                              ):

    train_file = ''

    word_counts = dict()
    
    for root, dirnames, filenames in os.walk(root_path):
        for filename in fnmatch.filter(filenames, '*.csv'):
            path = "csv/oneLineCode.csv"
            train_file = path
            dataframe = pd.read_csv(path, na_filter = False)
            for i,data in dataframe.iterrows():
                d = splitComment(data["comment"])
                c = tokenizeJavaCode(data["code"])
#                 print(i) # row number
#                 print("comment:")
#                 print(d) # comment words
                d = padding(d)
#                 print("code:")
#                 print(c) # code words
                c = padding(c)
                s = []
                s =  d+[SPL]+c # comment and code together
#                 print(s)
                add_counts(word_counts, s)
            with open(path, 'r', encoding='utf-8') as text:
                for line in text:
                    add_counts(word_counts, line)

    vocabulary = build_vocabulary(word_counts)
#     print("-- VOCABULARY BEGIN --")
#     print(vocabulary)
#     print("-- VOCABULARY END --")
    write_vocabulary(vocabulary, vocabFile)

    write_processed_dataset(train_file, train_output,vocabFile, combine)
#     write_processed_dataset(dev_txt_files, dev_output)
#     write_processed_dataset(test_txt_files, test_output)

def write_processed_dataset(input_file, output_file, vocabFile, combine):
    df = pd.DataFrame()
    word_vocabulary = read_vocabulary(vocabFile)
    dataframe = pd.read_csv(input_file, na_filter = False)
    for i,d in dataframe.iterrows():
        comment = []
        code = []
        label = 1 if d["non-information"] == "yes" else 0
        for token in padding(splitComment(d["comment"])):
            comment.append(word_vocabulary.get(token,0))
        for token in padding(tokenizeJavaCode(d["code"])):
            code.append(word_vocabulary.get(token,0))
        if combine:
            data = comment + [1] +code
            print(data)
            da = [{
                "data":data,
                "label":label,
            }]
            
            df = df.append(da,ignore_index=True,sort=False)
        else:
            da = [{
                "comment":comment,
                "code":code,
                "label":label,
            }]
            print(da)
            df = df.append(da,ignore_index=True,sort=False)

    df.to_csv(output_file, index=False)

In [4]:
delimiters = "#", ".",",","<b>","</b>","-",":","<br>","_","?"," ",";"
def splitComment(string,delimiters = delimiters, maxsplit=0):
## replace all web https to https
    for l in string.split():
        if l.startswith("https"):
            string = string.replace(l,"https")
    ## split string by delimiters
    regexPattern = '|'.join(map(re.escape, delimiters))
    result =  re.split(regexPattern, string, maxsplit)
    ## split string by uppercase
    f = []
    for r in result:
        p =  re.sub(r'((?<=[a-z])[A-Z]|(?<!\A)[A-Z](?=[a-z]))', r' \1', r)
#         print(p)
        for q in p.split():
            if q != "":
                f.append(q)
    return f
def tokenizeJavaCode(code):
    result = []
    try:
        tokens = list(javalang.tokenizer.tokenize(code))
        for token in tokens:
            result.append(token.value)
        return result
    except:
        return list(code)
#TODO stopwords
STOPWORDS_RAW=["(?i)or$", "(?i)and$", "(?i)i$"]

def iterable_to_dict(arr):
#     print("-- DEBUG BEGIN --")
#     print(dict((x.strip(), i) for (i, x) in enumerate(arr)))
#     print("-- DEBUG END --")
    return dict((x.strip(), (i)) for (i, x) in enumerate(arr))

def read_vocabulary(file_name):
    with open(file_name, 'r', encoding='utf-8') as f:
        return iterable_to_dict(f.readlines())
def add_counts(word_counts, line):
    stopreg = []
    for rawregex in STOPWORDS_RAW:
        stopreg.append(re.compile(rawregex))
#     print(stopreg)
    for w in line:  
        if any(regex.match(w) for regex in stopreg):
#             print("Stopword:",w)
            continue
        word_counts[w] = word_counts.get(w, 0) + 1
END = "</S>"
SPL = "<SPL>"
PAD = "<PAD>"
# NUM = "<NUM>"
UNK = "<NUK>"
MAX_WORD_VOCABULARY_SIZE = 100000
MIN_WORD_COUNT_IN_VOCAB = 1
MAX_SEQUENCE_LEN = 20


def padding(sentences, padding_size = MAX_SEQUENCE_LEN):
    result = []
    if len(sentences) >= padding_size:
        result = sentences[:padding_size]
    else:
        result = sentences
        for i in range(padding_size-len(sentences)):
            result.append(PAD)
    return result

def dump(d, path):
    with open(path, 'w') as f:
        for s in d:
            f.write("%s\n" % repr(s))
        
def write_vocabulary(vocabulary, file_name):
    vocabulary.insert(0,SPL)
    vocabulary.insert(0,UNK)
    if END not in vocabulary:
        vocabulary.append(END)

    print("Vocabulary size (write_vocabulary): %d" % len(vocabulary))

    with open(file_name, 'w', encoding='utf-8') as f:
        f.write("\n".join(vocabulary))
        
def build_vocabulary(word_counts):
    return [wc[0] for wc in reversed(sorted(word_counts.items(), key=operator.itemgetter(1))) if wc[1] >= MIN_WORD_COUNT_IN_VOCAB and wc[0] != UNK][:MAX_WORD_VOCABULARY_SIZE] # Unk will be appended to end

In [5]:
root_path = "csv"
vocabFile = "csv/vocab.txt"
create_dev_test_train_split_and_vocabulary(root_path,"csv/training_data.csv",vocabFile,True)

Vocabulary size (write_vocabulary): 3843
[3842, 1648, 630, 115, 121, 3841, 1647, 458, 457, 629, 3840, 297, 1646, 3839, 250, 535, 3838, 457, 534, 3837, 1, 70, 114, 88, 1645, 19, 1644, 774, 12, 628, 1643, 12, 1642, 4, 21, 38, 2, 2, 2, 2, 2]
[148, 133, 533, 532, 3836, 245, 456, 113, 1641, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 455, 18, 197, 19, 627, 18, 1640, 12, 148, 21, 37, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[3835, 172, 80, 42, 1027, 626, 163, 137, 42, 1639, 3834, 1026, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1025, 18, 3833, 19, 3832, 19, 21, 12, 3831, 12, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[1024, 78, 42, 147, 1023, 625, 59, 773, 42, 123, 146, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 100, 141, 3830, 19, 454, 214, 21, 38, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[192, 264, 111, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 3829, 19, 187, 18, 3828, 21, 12, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[213, 78, 171, 75, 7, 140, 127, 137, 42, 405, 144, 166, 139, 97, 75, 7, 1638, 80, 98, 127, 1, 296, 624, 52, 108, 37, 2, 2, 2, 2, 2, 

[3609, 953, 161, 144, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 78, 19, 103, 3608, 18, 3607, 19, 144, 21, 21, 38, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[593, 1525, 735, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 78, 19, 103, 3606, 19, 21, 21, 38, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[1524, 3605, 137, 42, 216, 235, 194, 205, 0, 734, 105, 59, 952, 193, 2, 2, 2, 2, 2, 2, 1, 733, 37, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[192, 168, 84, 951, 111, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 3604, 19, 187, 18, 3603, 21, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[3602, 42, 152, 261, 59, 42, 165, 232, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 70, 88, 950, 19, 3601, 3600, 21, 38, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[217, 7, 140, 303, 592, 240, 105, 42, 59, 949, 96, 80, 42, 600, 392, 2, 2, 2, 2, 2, 1, 70, 114, 50, 32, 48, 604, 50, 32, 48, 173, 19, 270, 50, 32, 48, 3599, 21, 38, 2, 2]
[109, 3598, 1523, 320, 277, 78, 42, 948, 288, 110, 732, 282, 3597, 947, 75, 3596, 161, 42

[1581, 105, 751, 181, 585, 105, 1467, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 311, 18, 335, 19, 1580, 21, 37, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[166, 119, 132, 1466, 59, 133, 353, 207, 79, 319, 398, 498, 7, 1465, 284, 119, 438, 505, 310, 332, 1, 1464, 52, 201, 37, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[312, 1463, 80, 603, 450, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 3488, 3487, 52, 258, 18, 3486, 19, 21, 37, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[352, 7, 82, 3485, 1462, 105, 42, 123, 92, 921, 166, 181, 212, 75, 275, 84, 121, 3484, 42, 140, 1, 70, 114, 88, 3483, 19, 243, 92, 21, 267, 385, 38, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[0, 3482, 580, 274, 59, 108, 75, 110, 448, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 274, 18, 1461, 19, 3481, 18, 3480, 19, 21, 21, 37, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[579, 920, 130, 109, 1460, 1459, 613, 97, 96, 183, 25, 3479, 75, 143, 115, 42, 255, 208, 3478, 178, 1, 70, 141, 3477, 19, 21, 38, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[3476, 42, 1458, 3475, 45

[1405, 212, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 185, 38, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[3374, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 106, 1404, 18, 3373, 19, 21, 37, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[238, 260, 78, 171, 75, 135, 350, 92, 127, 620, 79, 97, 566, 2, 2, 2, 2, 2, 2, 2, 1, 70, 141, 3372, 19, 83, 566, 21, 38, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[746, 698, 3371, 719, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 106, 201, 37, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[3370, 133, 3369, 3368, 3367, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[889, 42, 176, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 185, 38, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[138, 215, 59, 113, 42, 219, 3366, 42, 565, 75, 7, 707, 1403, 2, 2, 2, 2, 2, 2, 2, 1, 434, 3365, 52, 19, 19, 3364, 21, 97, 18, 1402, 18, 3363, 19, 21, 21, 2, 2, 2, 

[225, 116, 302, 686, 98, 720, 563, 740, 159, 164, 319, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 112, 50, 83, 48, 1016, 52, 82, 495, 50, 48, 19, 21, 37, 2, 2, 2, 2, 2, 2, 2]
[3269, 101, 132, 3268, 481, 202, 97, 277, 560, 285, 3267, 420, 2, 2, 2, 2, 2, 2, 2, 2, 1, 280, 50, 122, 12, 683, 48, 101, 52, 3266, 19, 419, 12, 682, 12, 3265, 21, 37, 2, 2, 2]
[79, 42, 3264, 101, 3263, 170, 59, 99, 149, 97, 3262, 3261, 98, 179, 188, 1363, 142, 3260, 59, 135, 1, 79, 19, 122, 93, 63, 101, 21, 38, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[41, 1497, 693, 681, 80, 42, 121, 555, 1362, 246, 3259, 3258, 203, 75, 401, 84, 7, 3257, 329, 2, 1, 70, 81, 1361, 175, 1360, 38, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[116, 524, 84, 336, 554, 3256, 105, 872, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 78, 19, 259, 18, 3255, 19, 21, 248, 3254, 18, 3253, 21, 38, 2, 2, 2, 2, 2, 2, 2]
[337, 110, 79, 97, 93, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 871, 18, 113, 19, 358, 21, 18, 162, 19, 1359, 21, 37, 2, 2, 2, 2, 2, 2, 2, 2]
[166, 1

...

[109, 1590, 259, 134, 99, 2234, 59, 42, 121, 760, 2233, 1098, 2, 2, 2, 2, 2, 2, 2, 2, 1, 77, 380, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[342, 443, 876, 1368, 80, 116, 357, 59, 2232, 2231, 246, 0, 328, 517, 1097, 2230, 875, 2229, 611, 52, 1, 72, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[468, 557, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 70, 114, 124, 83, 2228, 52, 2227, 37, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[160, 59, 260, 84, 276, 42, 212, 433, 110, 99, 666, 4, 22, 78, 42, 92, 75, 110, 235, 160, 1, 307, 141, 2226, 37, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[169, 250, 208, 2225, 79, 286, 108, 2224, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 258, 18, 2223, 19, 21, 18, 755, 19, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[1004, 42, 143, 101, 0, 1426, 340, 137, 42, 143, 310, 306, 512, 2222, 396, 104, 75, 2221, 59, 42, 1, 70, 81, 2220, 38, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[95, 81, 854, 7, 127, 80, 350, 150, 98, 190, 99, 361, 59, 1

[1081, 232, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2115, 18, 162, 19, 82, 2114, 19, 423, 12, 214, 18, 1080, 19, 21, 12, 548, 12, 2, 2, 2]
[642, 466, 109, 878, 918, 161, 42, 252, 1404, 134, 133, 7, 1079, 466, 105, 42, 880, 142, 95, 1079, 1, 2113, 52, 423, 18, 504, 19, 21, 18, 1148, 19, 21, 37, 2, 2, 2, 2, 2, 2, 2, 2]
[657, 59, 2112, 2111, 965, 79, 143, 2110, 161, 565, 223, 0, 1078, 285, 2, 2, 2, 2, 2, 2, 1, 70, 81, 2109, 126, 315, 38, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[365, 79, 350, 92, 326, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 70, 81, 2108, 126, 598, 50, 744, 48, 38, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[2107, 868, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1077, 2106, 52, 82, 1077, 19, 21, 37, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[95, 81, 75, 801, 7, 878, 431, 121, 645, 138, 698, 202, 121, 645, 452, 334, 104, 399, 2105, 247, 1, 70, 237, 81, 262, 50, 73, 48, 38, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[896, 42, 268, 137, 42, 621, 80, 42, 123, 9

[558, 1997, 505, 313, 0, 290, 382, 519, 1996, 421, 80, 382, 59, 328, 427, 381, 1995, 2, 2, 2, 1, 382, 18, 1994, 19, 1439, 19, 21, 21, 37, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[362, 316, 661, 42, 547, 75, 110, 133, 432, 139, 812, 660, 97, 2, 2, 2, 2, 2, 2, 2, 1, 1273, 19, 208, 12, 199, 21, 37, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[292, 149, 59, 441, 98, 42, 819, 818, 59, 133, 7, 353, 207, 98, 75, 110, 424, 84, 42, 152, 1, 81, 1245, 126, 292, 38, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[1993, 7, 179, 1992, 203, 98, 1991, 1247, 218, 42, 123, 1065, 2, 2, 2, 2, 2, 2, 2, 2, 1, 50, 73, 48, 425, 50, 158, 48, 1990, 19, 262, 50, 73, 48, 203, 12, 892, 1065, 12, 1591, 1989]
[192, 527, 111, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1988, 19, 187, 18, 1987, 21, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[95, 304, 670, 211, 99, 149, 84, 7, 208, 0, 325, 1986, 84, 42, 93, 253, 243, 112, 365, 0, 1, 70, 81, 1985, 126, 315, 38, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[1984, 1983, 120, 2, 2

[1883, 42, 143, 101, 115, 42, 146, 298, 1001, 166, 201, 42, 147, 134, 113, 1075, 78, 1140, 1023, 625, 1, 70, 88, 438, 19, 141, 1001, 21, 38, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[95, 96, 75, 233, 115, 159, 164, 468, 78, 7, 146, 699, 199, 75, 1882, 161, 42, 147, 691, 42, 1, 70, 88, 1881, 19, 124, 346, 212, 21, 38, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[95, 96, 75, 233, 115, 159, 164, 468, 183, 42, 147, 625, 59, 173, 7, 82, 93, 298, 848, 205, 1, 70, 88, 1070, 19, 124, 122, 637, 21, 38, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[160, 198, 317, 476, 79, 97, 146, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 100, 88, 1880, 19, 21, 38, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[402, 42, 93, 253, 125, 42, 1314, 239, 84, 42, 671, 469, 166, 135, 93, 253, 256, 272, 301, 971, 1, 70, 88, 1879, 19, 122, 93, 21, 38, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[95, 96, 836, 42, 123, 93, 0, 1878, 104, 206, 268, 84, 42, 356, 166, 135, 93, 365, 75, 301, 1, 70, 88, 1877, 19, 124, 122, 637, 21, 38, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


[1185, 135, 1782, 1781, 42, 479, 59, 441, 42, 146, 170, 265, 2, 2, 2, 2, 2, 2, 2, 2, 1, 439, 18, 1355, 19, 21, 37, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[593, 42, 180, 516, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1780, 1779, 52, 1778, 18, 1777, 19, 151, 18, 157, 18, 1776, 19, 21, 21, 37, 2, 2, 2, 2]
[1142, 162, 82, 93, 59, 42, 143, 191, 512, 133, 80, 1775, 2, 2, 2, 2, 2, 2, 2, 2, 1, 78, 19, 151, 18, 157, 18, 713, 19, 266, 18, 1774, 21, 21, 38, 2, 2, 2, 2, 2, 2]
[1773, 211, 149, 59, 247, 42, 550, 1772, 87, 1771, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1361, 18, 1364, 19, 19, 21, 230, 439, 18, 1770, 19, 21, 18, 1769, 19, 21, 21, 37, 2, 2]
[513, 223, 0, 1078, 235, 84, 42, 1768, 0, 1767, 232, 803, 79, 1059, 804, 84, 42, 1766, 129, 125, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[1765, 79, 1764, 360, 82, 1763, 1762, 0, 1761, 1760, 149, 881, 42, 159, 164, 414, 1759, 2, 2, 2, 1, 70, 81, 1758, 38, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[95, 96, 831, 1757, 8